# Survival analysis para la empresa JAMPP - Notebook usado para el installs. Estudio comportamiental, analisis de los valores promedios por usuario para usar esos valores en el target set


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import lifelines
import category_encoders as ce
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score
import xgboost

In [2]:
events = pd.read_csv('./events.csv')

/Users/yorickvinesse/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
clicks = pd.read_csv('./clicks.csv')

/Users/yorickvinesse/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
installs = pd.read_csv('./installs.csv')

/Users/yorickvinesse/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Que datos nos conviene usar?

Acá estamos haciendo un _survival analysis_ entonces se necesita mostrar el momento al cual ocurrió un evento. En nuestro caso, si un evento aparece en el dataset de _auctions_, es porque ocurrió. El survival analysis se enfoca en la primera vez que ocurre un evento, puesto que hace una regresión de un set de caractéristicas en frente a una fecha de ocurrencia del evento.

Pero es cierto que muchos usuarios, aca representados por sus *device_id* aparecen varias veces en la lista de subastas (_auctions_), considerando que 475338 *device_id* únicos se pueden contar adentro del set de _auctions_ que almacena 47409528 observaciones distintas (100 veces más).

Dado eso, parecería inoportuno no considerar la característica de repetición de aparición de los usuarios cuando hacemos la predicción de su reaparición más adelante.

Podemos tratar de analizar datos como la cantidad de clicks hecha por un usuario antes de llevar a cabo una instalación, o la cantidad de veces que apareció en una subasta.

### Para aliviar los sets, sacamos las columnas que llevan el mismo valor para todas filas.

In [5]:
events.drop(columns = ['device_countrycode'], inplace = True)
clicks.drop(columns = ['country_code'], inplace = True)
installs.drop(columns = ['device_countrycode'], inplace = True)

# Cuandó aparecerá un usuario dado?

### Desarrollo alrededor del dataset _auctions_

Vamos a enriquecer el dataset relacionado a las subastas incluyendo, para cada observación, la cantidad de veces que ocurrió una aparición del mismo *device_id* dentro de la ventana de estudio anterior. También vamos a indicar el tiempo que pasó desde la última vez que ocurrió el evento para el mismo *device_id*.

Primero, queremos tener un modelo que nos indica cuando será la próxima vez que aparecerá un usuario en una subasta.

Es necesario convertir los tiempos de ocurrencia en segundos para que nos de una duración hasta el evento.

## Análisis de los eventos y installs para tratar de predecir con un modelo de Cox.

Se debe tratar de predecir cuales features nos van a interesar en el fit. El tipo de conexión, por ejemplo, servirá mucho, mientras el countrycode no (solo hay uno en el set de datos).

In [6]:
eventsCurated = events.drop(columns = ['ref_type', 'session_user_agent', 'device_os_version', 'device_brand',\
                                       'device_model', 'trans_id', 'user_agent', 'ip_address'])

In [7]:
eventsCurated.head(2)

,index,date,event_id,ref_hash,application_id,attributed,device_city,event_uuid,carrier,kind,device_os,wifi,connection_type,device_language
0,2130678,2019-04-20 01:42:49.120,0,5857744372586891366,210,False,NaN,5b506964-5f47-4b28-a8c2-8a92d6c23379,NaN,5.882882e+18,NaN,False,NaN,3.301378e+18
1,2130680,2019-04-20 01:42:49.340,1,7642521036780133571,210,False,NaN,f1fb9d15-1a7b-4116-8d3b-c4c403e197e2,NaN,4.017674e+18,NaN,False,NaN,NaN


## Vamos a curar los features de installs ahora, dejar todas las columnas que ya aparecen en _events_ y despues empezar a mergear los dos dataframes, antes de separarlos en ventanas de tres días.

In [125]:
installs.count()

created               481511
application_id        481511
ref_type              481511
ref_hash              481511
click_hash              1142
attributed            481511
implicit              481511
device_brand          276443
device_model          454619
session_user_agent    466672
user_agent            330768
event_uuid            103168
kind                  103168
wifi                  294829
trans_id                8933
ip_address            481511
device_language       453934
dtype: int64

In [8]:
installsCurated = installs.drop(columns = ['created', 'application_id', 'ref_type', 'ref_hash', \
                                           'click_hash', 'attributed', 'device_brand', 'device_model',\
                                           'session_user_agent', 'user_agent', 'kind', 'wifi', 'trans_id',\
                                          'ip_address', 'device_language'])

### Nos quedamos con los implicit = True, para solo analizar los installs de los cuales tenemos el evento registrado.

In [9]:
installsCurated = installsCurated[installsCurated.implicit]

In [10]:
installsCurated['installed'] = 1

In [11]:
installsCurated.head(2)

,implicit,event_uuid,installed
0,True,79837499-2f2a-4605-a663-e322f759424f,1
2,True,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,1


outer join

In [12]:
eventsAndInstallsCurated = pd.merge(eventsCurated, installsCurated, on = 'event_uuid', how = 'left')

In [13]:
eventsAndInstallsCurated.count()

index              7744666
date               7744666
event_id           7744666
ref_hash           7744666
application_id     7744666
attributed         7744666
device_city        1895020
event_uuid         7714894
carrier            1925986
kind               7714894
device_os          1870190
wifi               7744666
connection_type    1809296
device_language    5665494
implicit            102359
installed           102359
dtype: int64

In [14]:
eventsAndInstallsCurated = eventsAndInstallsCurated.drop(columns = ['implicit'])

### Aca llenamos la columna de _installed_ con 0 en los eventos que no llevan a una instalacion. Eso sera la columna indicando el _censorship_ para nuestro modelo Cox

In [15]:
eventsAndInstallsCurated[['installed']] = eventsAndInstallsCurated[['installed']].fillna(value = 0)

In [16]:
eventsAndInstallsCurated.count()

index              7744666
date               7744666
event_id           7744666
ref_hash           7744666
application_id     7744666
attributed         7744666
device_city        1895020
event_uuid         7714894
carrier            1925986
kind               7714894
device_os          1870190
wifi               7744666
connection_type    1809296
device_language    5665494
installed          7744666
dtype: int64

# Separacion por ventana, para no tratar con sets de datos enormes. Asi empezamos las pruebas y si nos anda bien, podemos hacerlo con todo.

In [16]:
eventsInstallsVentanaUno = eventsAndInstallsCurated.loc[(eventsAndInstallsCurated['date'] < "2019-04-21")]
eventsInstallsVentanaUno = eventsInstallsVentanaUno.loc[(eventsAndInstallsCurated['date'] > "2019-04-18")]

In [17]:
eventsInstallsVentanaDos = eventsAndInstallsCurated.loc[(eventsAndInstallsCurated['date'] < "2019-04-22")]
eventsInstallsVentanaDos = eventsInstallsVentanaDos.loc[(eventsAndInstallsCurated['date'] > "2019-04-19")]

In [18]:
eventsInstallsVentanaTres = eventsAndInstallsCurated.loc[(eventsAndInstallsCurated['date'] < "2019-04-23")]
eventsInstallsVentanaTres = eventsInstallsVentanaTres.loc[(eventsAndInstallsCurated['date'] > "2019-04-20")]

In [19]:
eventsInstallsVentanaCuatro = eventsAndInstallsCurated.loc[(eventsAndInstallsCurated['date'] < "2019-04-24")]
eventsInstallsVentanaCuatro = eventsInstallsVentanaCuatro.loc[(eventsAndInstallsCurated['date'] > "2019-04-21")]

In [20]:
eventsInstallsVentanaCinco = eventsAndInstallsCurated.loc[(eventsAndInstallsCurated['date'] < "2019-04-25")]
eventsInstallsVentanaCinco = eventsInstallsVentanaCinco.loc[(eventsAndInstallsCurated['date'] > "2019-04-22")]

### Tratamos de ver que importancia tiene el uso del wifi en la instalación de apps. Damos a cada device_id como feature su promedio de uso del wifi en los eventos en los cuales participó. 

### Hacemos pruebas con la ventana uno.

In [21]:
proporcionWifiPorDeviceVentanaUno = eventsInstallsVentanaUno[['ref_hash', 'wifi']].groupby('ref_hash').mean()
proporcionWifiPorDeviceVentanaDos = eventsInstallsVentanaDos[['ref_hash', 'wifi']].groupby('ref_hash').mean()
proporcionWifiPorDeviceVentanaTres = eventsInstallsVentanaTres[['ref_hash', 'wifi']].groupby('ref_hash').mean()
proporcionWifiPorDeviceVentanaCuatro = eventsInstallsVentanaCuatro[['ref_hash', 'wifi']].groupby('ref_hash').mean()
proporcionWifiPorDeviceVentanaCinco = eventsInstallsVentanaCinco[['ref_hash', 'wifi']].groupby('ref_hash').mean()

In [22]:
proporcionWifiPorDeviceVentanaUno.head(2)

,wifi
ref_hash,
41863526108385,0.0
161514654074162,1.0


In [23]:
proporcionWifiPorDeviceVentanaUno = proporcionWifiPorDeviceVentanaUno.stack().reset_index()
proporcionWifiPorDeviceVentanaDos = proporcionWifiPorDeviceVentanaDos.stack().reset_index()
proporcionWifiPorDeviceVentanaTres = proporcionWifiPorDeviceVentanaTres.stack().reset_index()
proporcionWifiPorDeviceVentanaCuatro = proporcionWifiPorDeviceVentanaCuatro.stack().reset_index()
proporcionWifiPorDeviceVentanaCinco = proporcionWifiPorDeviceVentanaCinco.stack().reset_index()

In [24]:
proporcionWifiPorDeviceVentanaUno = proporcionWifiPorDeviceVentanaUno.drop(columns = ['level_1'])
proporcionWifiPorDeviceVentanaDos = proporcionWifiPorDeviceVentanaDos.drop(columns = ['level_1'])
proporcionWifiPorDeviceVentanaTres = proporcionWifiPorDeviceVentanaTres.drop(columns = ['level_1'])
proporcionWifiPorDeviceVentanaCuatro = proporcionWifiPorDeviceVentanaCuatro.drop(columns = ['level_1'])
proporcionWifiPorDeviceVentanaCinco = proporcionWifiPorDeviceVentanaCinco.drop(columns = ['level_1'])

In [25]:
proporcionWifiPorDeviceVentanaUno.columns = ['ref_hash', 'proporcion_wifi']
proporcionWifiPorDeviceVentanaDos.columns = ['ref_hash', 'proporcion_wifi']
proporcionWifiPorDeviceVentanaTres.columns = ['ref_hash', 'proporcion_wifi']
proporcionWifiPorDeviceVentanaCuatro.columns = ['ref_hash', 'proporcion_wifi']
proporcionWifiPorDeviceVentanaCinco.columns = ['ref_hash', 'proporcion_wifi']

### Nos queremos quedar con un porcentaje, mas legible.

In [26]:
proporcionWifiPorDeviceVentanaUno['proporcion_wifi'] = proporcionWifiPorDeviceVentanaUno['proporcion_wifi']*100
proporcionWifiPorDeviceVentanaDos['proporcion_wifi'] = proporcionWifiPorDeviceVentanaDos['proporcion_wifi']*100
proporcionWifiPorDeviceVentanaTres['proporcion_wifi'] = proporcionWifiPorDeviceVentanaTres['proporcion_wifi']*100
proporcionWifiPorDeviceVentanaCuatro['proporcion_wifi'] = proporcionWifiPorDeviceVentanaCuatro['proporcion_wifi']*100
proporcionWifiPorDeviceVentanaCinco['proporcion_wifi'] = proporcionWifiPorDeviceVentanaCinco['proporcion_wifi']*100

In [27]:
proporcionWifiPorDeviceVentanaUno.head(2)

,ref_hash,proporcion_wifi
0,41863526108385,0.0
1,161514654074162,100.0


In [28]:
eventsInstallsVentanaUno = pd.merge(eventsInstallsVentanaUno, proporcionWifiPorDeviceVentanaUno,\
                                    on = "ref_hash", how = "left")
eventsInstallsVentanaDos = pd.merge(eventsInstallsVentanaDos, proporcionWifiPorDeviceVentanaDos,\
                                    on = "ref_hash", how = "left")
eventsInstallsVentanaTres = pd.merge(eventsInstallsVentanaTres, proporcionWifiPorDeviceVentanaTres,\
                                    on = "ref_hash", how = "left")
eventsInstallsVentanaCuatro = pd.merge(eventsInstallsVentanaCuatro, proporcionWifiPorDeviceVentanaCuatro,\
                                    on = "ref_hash", how = "left")
eventsInstallsVentanaCinco = pd.merge(eventsInstallsVentanaCinco, proporcionWifiPorDeviceVentanaCinco,\
                                    on = "ref_hash", how = "left")

In [29]:
eventsInstallsVentanaUno.head(2)

,index,date,event_id,ref_hash,application_id,attributed,device_city,event_uuid,carrier,kind,device_os,wifi,connection_type,device_language,installed,proporcion_wifi
0,2130678,2019-04-20 01:42:49.120,0,5857744372586891366,210,False,NaN,5b506964-5f47-4b28-a8c2-8a92d6c23379,NaN,5.882882e+18,NaN,False,NaN,3.301378e+18,0.0,0.0
1,2130680,2019-04-20 01:42:49.340,1,7642521036780133571,210,False,NaN,f1fb9d15-1a7b-4116-8d3b-c4c403e197e2,NaN,4.017674e+18,NaN,False,NaN,NaN,0.0,0.0


## Tratamiento de la fecha y de la forma del tiempo. Hacemos como lo que hicimos para auctions.

In [30]:
eventsInstallsVentanaUno['fechaInicial'] = pd.to_datetime("2019-04-18 00:00:00")
eventsInstallsVentanaDos['fechaInicial'] = pd.to_datetime("2019-04-19 00:00:00")
eventsInstallsVentanaTres['fechaInicial'] = pd.to_datetime("2019-04-20 00:00:00")
eventsInstallsVentanaCuatro['fechaInicial'] = pd.to_datetime("2019-04-21 00:00:00")
eventsInstallsVentanaCinco['fechaInicial'] = pd.to_datetime("2019-04-22 00:00:00")

In [31]:
eventsInstallsVentanaUno["sum"] = 1
eventsInstallsVentanaDos["sum"] = 1
eventsInstallsVentanaTres["sum"] = 1
eventsInstallsVentanaCuatro["sum"] = 1
eventsInstallsVentanaCinco["sum"] = 1

In [32]:
eventsInstallsVentanaUno.loc[:,'date'] = pd.to_datetime(eventsInstallsVentanaUno['date'], errors = 'coerce')
eventsInstallsVentanaDos.loc[:,'date'] = pd.to_datetime(eventsInstallsVentanaDos['date'], errors = 'coerce')
eventsInstallsVentanaTres.loc[:,'date'] = pd.to_datetime(eventsInstallsVentanaTres['date'], errors = 'coerce')
eventsInstallsVentanaCuatro.loc[:,'date'] = pd.to_datetime(eventsInstallsVentanaCuatro['date'], errors = 'coerce')
eventsInstallsVentanaCinco.loc[:,'date'] = pd.to_datetime(eventsInstallsVentanaCinco['date'], errors = 'coerce')

In [33]:
eventsInstallsVentanaUno["date"] = eventsInstallsVentanaUno["date"] - eventsInstallsVentanaUno["fechaInicial"]
eventsInstallsVentanaDos["date"] = eventsInstallsVentanaDos["date"] - eventsInstallsVentanaDos["fechaInicial"]
eventsInstallsVentanaTres["date"] = eventsInstallsVentanaTres["date"] - eventsInstallsVentanaTres["fechaInicial"]
eventsInstallsVentanaCuatro["date"] = eventsInstallsVentanaCuatro["date"] - eventsInstallsVentanaCuatro["fechaInicial"]
eventsInstallsVentanaCinco["date"] = eventsInstallsVentanaCinco["date"] - eventsInstallsVentanaCinco["fechaInicial"]

In [34]:
eventsInstallsVentanaUno.head(2)

,index,date,event_id,ref_hash,application_id,attributed,device_city,event_uuid,carrier,kind,device_os,wifi,connection_type,device_language,installed,proporcion_wifi,fechaInicial,sum
0,2130678,2 days 01:42:49.120000,0,5857744372586891366,210,False,NaN,5b506964-5f47-4b28-a8c2-8a92d6c23379,NaN,5.882882e+18,NaN,False,NaN,3.301378e+18,0.0,0.0,2019-04-18,1
1,2130680,2 days 01:42:49.340000,1,7642521036780133571,210,False,NaN,f1fb9d15-1a7b-4116-8d3b-c4c403e197e2,NaN,4.017674e+18,NaN,False,NaN,NaN,0.0,0.0,2019-04-18,1


### Convertimos el tiempo en cantidad de segundos.

In [35]:
eventsInstallsVentanaUno["date"] = eventsInstallsVentanaUno["date"].dt.total_seconds()
eventsInstallsVentanaDos["date"] = eventsInstallsVentanaDos["date"].dt.total_seconds()
eventsInstallsVentanaTres["date"] = eventsInstallsVentanaTres["date"].dt.total_seconds()
eventsInstallsVentanaCuatro["date"] = eventsInstallsVentanaCuatro["date"].dt.total_seconds()
eventsInstallsVentanaCinco["date"] = eventsInstallsVentanaCinco["date"].dt.total_seconds()

### Ya no necesitamos la fecha inicial

In [36]:
eventsInstallsVentanaUno = eventsInstallsVentanaUno.drop(columns = ['fechaInicial'])
eventsInstallsVentanaDos = eventsInstallsVentanaDos.drop(columns = ['fechaInicial'])
eventsInstallsVentanaTres = eventsInstallsVentanaTres.drop(columns = ['fechaInicial'])
eventsInstallsVentanaCuatro = eventsInstallsVentanaCuatro.drop(columns = ['fechaInicial'])
eventsInstallsVentanaCuatro = eventsInstallsVentanaCinco.drop(columns = ['fechaInicial'])

### Contamos la cantidad de eventos por device_id (o ref_hash) en cada ventana

In [37]:
cantidadDeEventosPorIdVentanaUno = eventsInstallsVentanaUno[['ref_hash','sum']].groupby('ref_hash').sum()
cantidadDeEventosPorIdVentanaDos = eventsInstallsVentanaDos[['ref_hash','sum']].groupby('ref_hash').sum()
cantidadDeEventosPorIdVentanaTres = eventsInstallsVentanaTres[['ref_hash','sum']].groupby('ref_hash').sum()
cantidadDeEventosPorIdVentanaCuatro = eventsInstallsVentanaCuatro[['ref_hash','sum']].groupby('ref_hash').sum()
cantidadDeEventosPorIdVentanaCinco = eventsInstallsVentanaCinco[['ref_hash','sum']].groupby('ref_hash').sum()

In [38]:
cantidadDeEventosPorIdVentanaUno = cantidadDeEventosPorIdVentanaUno.stack().reset_index()
cantidadDeEventosPorIdVentanaDos = cantidadDeEventosPorIdVentanaDos.stack().reset_index()
cantidadDeEventosPorIdVentanaTres = cantidadDeEventosPorIdVentanaTres.stack().reset_index()
cantidadDeEventosPorIdVentanaCuatro = cantidadDeEventosPorIdVentanaCuatro.stack().reset_index()
cantidadDeEventosPorIdVentanaCinco = cantidadDeEventosPorIdVentanaCinco.stack().reset_index()

In [39]:
cantidadDeEventosPorIdVentanaUno = cantidadDeEventosPorIdVentanaUno.drop(columns = ['level_1'])
cantidadDeEventosPorIdVentanaDos = cantidadDeEventosPorIdVentanaDos.drop(columns = ['level_1'])
cantidadDeEventosPorIdVentanaTres = cantidadDeEventosPorIdVentanaTres.drop(columns = ['level_1'])
cantidadDeEventosPorIdVentanaCuatro = cantidadDeEventosPorIdVentanaCuatro.drop(columns = ['level_1'])
cantidadDeEventosPorIdVentanaCinco = cantidadDeEventosPorIdVentanaCinco.drop(columns = ['level_1'])

In [40]:
cantidadDeEventosPorIdVentanaUno.columns = ['ref_hash', 'cantidad_eventos']
cantidadDeEventosPorIdVentanaDos.columns = ['ref_hash', 'cantidad_eventos']
cantidadDeEventosPorIdVentanaTres.columns = ['ref_hash', 'cantidad_eventos']
cantidadDeEventosPorIdVentanaCuatro.columns = ['ref_hash', 'cantidad_eventos']
cantidadDeEventosPorIdVentanaCinco.columns = ['ref_hash', 'cantidad_eventos']

In [41]:
cantidadDeEventosPorIdVentanaDos.head(2)

,ref_hash,cantidad_eventos
0,40621409780134,6
1,41863526108385,72


### Vamos a tomar todos los eventos que no llegan a un install y pasar la fecha a 3 days. Eso llevará a muchas columnas iguales, entonces iremos suprimiendolas. Sin embargo, se almacenerá la cantidad total de eventos para cada device_id. 

#### Para hacer eso, vamos a aplicar un _groupby_ por device_id sumando la columna _installed_. Los que tienen un puntaje de 0 son los que hay que reducir. 


#### Paras los eventos que llegan a varios installs, se conservará uno solo con la primera fecha de ocurrencia, pero se almacenará el tiempo promedio entre dos installs.

### Contamos la cantidad de installs por device_id (o ref_hash) en cada ventana

In [42]:
cantidadDeInstallsPorIdVentanaUno = eventsInstallsVentanaUno[['ref_hash','installed']].groupby('ref_hash').sum()
cantidadDeInstallsPorIdVentanaDos = eventsInstallsVentanaDos[['ref_hash','installed']].groupby('ref_hash').sum()
cantidadDeInstallsPorIdVentanaTres = eventsInstallsVentanaTres[['ref_hash','installed']].groupby('ref_hash').sum()
cantidadDeInstallsPorIdVentanaCuatro=eventsInstallsVentanaCuatro[['ref_hash','installed']].groupby('ref_hash').sum()
cantidadDeInstallsPorIdVentanaCinco = eventsInstallsVentanaCinco[['ref_hash','installed']].groupby('ref_hash').sum()

In [43]:
cantidadDeInstallsPorIdVentanaUno = cantidadDeInstallsPorIdVentanaUno.stack().reset_index()
cantidadDeInstallsPorIdVentanaDos = cantidadDeInstallsPorIdVentanaDos.stack().reset_index()
cantidadDeInstallsPorIdVentanaTres = cantidadDeInstallsPorIdVentanaTres.stack().reset_index()
cantidadDeInstallsPorIdVentanaCuatro = cantidadDeInstallsPorIdVentanaCuatro.stack().reset_index()
cantidadDeInstallsPorIdVentanaCinco = cantidadDeInstallsPorIdVentanaCinco.stack().reset_index()

In [44]:
cantidadDeInstallsPorIdVentanaUno = cantidadDeInstallsPorIdVentanaUno.drop(columns = ['level_1'])
cantidadDeInstallsPorIdVentanaDos = cantidadDeInstallsPorIdVentanaDos.drop(columns = ['level_1'])
cantidadDeInstallsPorIdVentanaTres = cantidadDeInstallsPorIdVentanaTres.drop(columns = ['level_1'])
cantidadDeInstallsPorIdVentanaCuatro = cantidadDeInstallsPorIdVentanaCuatro.drop(columns = ['level_1'])
cantidadDeInstallsPorIdVentanaCinco = cantidadDeInstallsPorIdVentanaCinco.drop(columns = ['level_1'])

In [45]:
cantidadDeInstallsPorIdVentanaUno.columns = ['ref_hash', 'cantidad_installs']
cantidadDeInstallsPorIdVentanaDos.columns = ['ref_hash', 'cantidad_installs']
cantidadDeInstallsPorIdVentanaTres.columns = ['ref_hash', 'cantidad_installs']
cantidadDeInstallsPorIdVentanaCuatro.columns = ['ref_hash', 'cantidad_installs']
cantidadDeInstallsPorIdVentanaCinco.columns = ['ref_hash', 'cantidad_installs']

In [46]:
cantidadDeInstallsPorIdVentanaDos.head(2)

,ref_hash,cantidad_installs
0,40621409780134,0.0
1,41863526108385,0.0


# Aca nos ponemos a calcular el tiempo promerdio entre dos installs para cada decice_id. Le ponemos un valor del primer tiempo de ocurrencia si es que apareció una sola vez. Iremos cambiando este parametro.

### Los que nunca aparecieron, le ponemos primero un tiempo de 3 días, y veremos como cambia el modelo si le dejamos un NaN

### Primero nos quedamos con los que llevan un install al menos.

In [47]:
tiempoEntreInstallsPorIdVentanaUno = eventsInstallsVentanaUno.sort_values(['ref_hash', 'date'], ascending = [True, True])
tiempoEntreInstallsPorIdVentanaDos = eventsInstallsVentanaDos.sort_values(['ref_hash', 'date'], ascending = [True, True])
tiempoEntreInstallsPorIdVentanaTres = eventsInstallsVentanaTres.sort_values(['ref_hash', 'date'], ascending = [True, True])
tiempoEntreInstallsPorIdVentanaCuatro = eventsInstallsVentanaCuatro.sort_values(['ref_hash', 'date'], ascending = [True, True])
tiempoEntreInstallsPorIdVentanaCinco = eventsInstallsVentanaCinco.sort_values(['ref_hash', 'date'], ascending = [True, True])

In [48]:
tiempoEntreInstallsPorIdVentanaUno = \
tiempoEntreInstallsPorIdVentanaUno[tiempoEntreInstallsPorIdVentanaUno.installed == 1]
tiempoEntreInstallsPorIdVentanaDos = \
tiempoEntreInstallsPorIdVentanaDos[tiempoEntreInstallsPorIdVentanaDos.installed == 1]
tiempoEntreInstallsPorIdVentanaTres = \
tiempoEntreInstallsPorIdVentanaTres[tiempoEntreInstallsPorIdVentanaTres.installed == 1]
tiempoEntreInstallsPorIdVentanaCuatro = \
tiempoEntreInstallsPorIdVentanaCuatro[tiempoEntreInstallsPorIdVentanaCuatro.installed == 1]
tiempoEntreInstallsPorIdVentanaCinco = \
tiempoEntreInstallsPorIdVentanaCinco[tiempoEntreInstallsPorIdVentanaCinco.installed == 1]

In [49]:
tiempoEntreInstallsPorIdVentanaUno = tiempoEntreInstallsPorIdVentanaUno[['date', 'ref_hash']]
tiempoEntreInstallsPorIdVentanaDos = tiempoEntreInstallsPorIdVentanaDos[['date', 'ref_hash']]
tiempoEntreInstallsPorIdVentanaTres = tiempoEntreInstallsPorIdVentanaTres[['date', 'ref_hash']]
tiempoEntreInstallsPorIdVentanaCuatro = tiempoEntreInstallsPorIdVentanaCuatro[['date', 'ref_hash']]
tiempoEntreInstallsPorIdVentanaCinco = tiempoEntreInstallsPorIdVentanaCinco[['date', 'ref_hash']]

In [50]:
tiempoEntreInstallsPorIdVentanaUno.head()

,date,ref_hash
269132,76310.326,41863526108385
876963,76636.531,41863526108385
326585,60042.839,186034136943920
378393,65400.392,558877640599287
783412,148990.929,989540301617336


### Método para calcular la diferencia de tiempo

In [51]:
def agregarTiempoPromedio(dataframe):
    dataframe['time_diff'] = 0

    for I in range(1,len(dataframe) - 1):
        if ((dataframe.iloc[I,1] != dataframe.iloc[I-1,1]) and (dataframe.iloc[I,1] != dataframe.iloc[I+1,1])):
            dataframe.iloc[I,2] = dataframe.iloc[I,0] #tiempo hasta ocurrencia
        elif (dataframe.iloc[I,0] > dataframe.iloc[I-1,0]) and (dataframe.iloc[I,1] == dataframe.iloc[I-1,1]):
            dataframe.iloc[I,2] = dataframe.iloc[I,0] - dataframe.iloc[I-1,0]
        else: 
            dataframe.iloc[I,2]=0
    return (dataframe)

In [52]:
tiempoEntreInstallsPorIdVentanaUno = agregarTiempoPromedio(tiempoEntreInstallsPorIdVentanaUno)

In [54]:
tiempoEntreInstallsPorIdVentanaDos = agregarTiempoPromedio(tiempoEntreInstallsPorIdVentanaDos)
tiempoEntreInstallsPorIdVentanaTres = agregarTiempoPromedio(tiempoEntreInstallsPorIdVentanaTres)
tiempoEntreInstallsPorIdVentanaCuatro = agregarTiempoPromedio(tiempoEntreInstallsPorIdVentanaCuatro)
tiempoEntreInstallsPorIdVentanaCinco = agregarTiempoPromedio(tiempoEntreInstallsPorIdVentanaCinco)

### Atención: necesidad de agregar una parte que trata el primero y el ultimo elemento de la ventana.

In [55]:
tiempoEntreInstallsPorIdVentanaUno.head()

,date,ref_hash,time_diff
269132,76310.326,41863526108385,0.000
876963,76636.531,41863526108385,326.205
326585,60042.839,186034136943920,60042.839
378393,65400.392,558877640599287,65400.392
783412,148990.929,989540301617336,148990.929


### Nos liberamos de los valores a 0 para tener un promedio válido.

In [56]:
tiempoEntreInstallsPorIdVentanaUno = \
tiempoEntreInstallsPorIdVentanaUno[tiempoEntreInstallsPorIdVentanaUno.time_diff != 0]
tiempoEntreInstallsPorIdVentanaDos = \
tiempoEntreInstallsPorIdVentanaDos[tiempoEntreInstallsPorIdVentanaDos.time_diff != 0]
tiempoEntreInstallsPorIdVentanaTres = \
tiempoEntreInstallsPorIdVentanaTres[tiempoEntreInstallsPorIdVentanaTres.time_diff != 0]
tiempoEntreInstallsPorIdVentanaCuatro = \
tiempoEntreInstallsPorIdVentanaCuatro[tiempoEntreInstallsPorIdVentanaCuatro.time_diff != 0]
tiempoEntreInstallsPorIdVentanaCinco = \
tiempoEntreInstallsPorIdVentanaCinco[tiempoEntreInstallsPorIdVentanaCinco.time_diff != 0]

### Calculo del tiempo promedio entre events.

In [57]:
tiempoEntreInstallsPorIdVentanaUno = \
tiempoEntreInstallsPorIdVentanaUno[['ref_hash', 'time_diff']].groupby('ref_hash').mean()
tiempoEntreInstallsPorIdVentanaDos = \
tiempoEntreInstallsPorIdVentanaDos[['ref_hash', 'time_diff']].groupby('ref_hash').mean()
tiempoEntreInstallsPorIdVentanaTres = \
tiempoEntreInstallsPorIdVentanaTres[['ref_hash', 'time_diff']].groupby('ref_hash').mean()
tiempoEntreInstallsPorIdVentanaCuatro = \
tiempoEntreInstallsPorIdVentanaCuatro[['ref_hash', 'time_diff']].groupby('ref_hash').mean()
tiempoEntreInstallsPorIdVentanaCinco = \
tiempoEntreInstallsPorIdVentanaCinco[['ref_hash', 'time_diff']].groupby('ref_hash').mean()

In [165]:
tiempoEntreInstallsPorIdVentanaUno.head()

,time_diff
ref_hash,
41863526108385,326.205
186034136943920,129600.000
558877640599287,129600.000
989540301617336,129600.000
1187457308005333,129600.000


In [58]:
tiempoEntreInstallsPorIdVentanaUno = tiempoEntreInstallsPorIdVentanaUno.stack().reset_index()
tiempoEntreInstallsPorIdVentanaUno = tiempoEntreInstallsPorIdVentanaUno.drop(columns = ['level_1'])
tiempoEntreInstallsPorIdVentanaDos = tiempoEntreInstallsPorIdVentanaDos.stack().reset_index()
tiempoEntreInstallsPorIdVentanaDos = tiempoEntreInstallsPorIdVentanaDos.drop(columns = ['level_1'])
tiempoEntreInstallsPorIdVentanaTres = tiempoEntreInstallsPorIdVentanaTres.stack().reset_index()
tiempoEntreInstallsPorIdVentanaTres = tiempoEntreInstallsPorIdVentanaTres.drop(columns = ['level_1'])
tiempoEntreInstallsPorIdVentanaCuatro = tiempoEntreInstallsPorIdVentanaCuatro.stack().reset_index()
tiempoEntreInstallsPorIdVentanaCuatro = tiempoEntreInstallsPorIdVentanaCuatro.drop(columns = ['level_1'])
tiempoEntreInstallsPorIdVentanaCinco = tiempoEntreInstallsPorIdVentanaCinco.stack().reset_index()
tiempoEntreInstallsPorIdVentanaCinco = tiempoEntreInstallsPorIdVentanaCinco.drop(columns = ['level_1'])

In [59]:
tiempoEntreInstallsPorIdVentanaUno.columns = ['ref_hash', 'tiempo_prom_entre_installs']
tiempoEntreInstallsPorIdVentanaDos.columns = ['ref_hash', 'tiempo_prom_entre_installs']
tiempoEntreInstallsPorIdVentanaTres.columns = ['ref_hash', 'tiempo_prom_entre_installs']
tiempoEntreInstallsPorIdVentanaCuatro.columns = ['ref_hash', 'tiempo_prom_entre_installs']
tiempoEntreInstallsPorIdVentanaCinco.columns = ['ref_hash', 'tiempo_prom_entre_installs']

In [60]:
tiempoEntreInstallsPorIdVentanaDos.head()

,ref_hash,tiempo_prom_entre_installs
0,989540301617336,62590.929
1,1440206761476269,101367.243
2,1719664649402429,194009.325
3,2493454867357963,255003.890
4,2706649710050643,23822.230


### Llegó el momento de mergear todo en un único dataframe. Primero con el timepo promedio entre installs.

In [61]:
eventsInstallsVentanaUno = pd.merge(eventsInstallsVentanaUno, tiempoEntreInstallsPorIdVentanaUno,\
                                    on = 'ref_hash', how = 'left')
eventsInstallsVentanaDos = pd.merge(eventsInstallsVentanaDos, tiempoEntreInstallsPorIdVentanaDos,\
                                    on = 'ref_hash', how = 'left')
eventsInstallsVentanaTres = pd.merge(eventsInstallsVentanaTres, tiempoEntreInstallsPorIdVentanaTres,\
                                    on = 'ref_hash', how = 'left')
eventsInstallsVentanaCuatro = pd.merge(eventsInstallsVentanaCuatro, tiempoEntreInstallsPorIdVentanaCuatro,\
                                    on = 'ref_hash', how = 'left')
eventsInstallsVentanaCinco = pd.merge(eventsInstallsVentanaCinco, tiempoEntreInstallsPorIdVentanaCinco,\
                                    on = 'ref_hash', how = 'left')

### Despues con la cantidad de installs y de eventos por device_id.

In [62]:
eventsInstallsVentanaUno = pd.merge(eventsInstallsVentanaUno, cantidadDeEventosPorIdVentanaUno,\
                                    on = 'ref_hash', how = 'left')
eventsInstallsVentanaDos = pd.merge(eventsInstallsVentanaDos, cantidadDeEventosPorIdVentanaDos,\
                                    on = 'ref_hash', how = 'left')
eventsInstallsVentanaTres = pd.merge(eventsInstallsVentanaTres, cantidadDeEventosPorIdVentanaTres,\
                                    on = 'ref_hash', how = 'left')
eventsInstallsVentanaCuatro = pd.merge(eventsInstallsVentanaCuatro, cantidadDeEventosPorIdVentanaCuatro,\
                                    on = 'ref_hash', how = 'left')
eventsInstallsVentanaCinco = pd.merge(eventsInstallsVentanaCinco, cantidadDeEventosPorIdVentanaCinco,\
                                    on = 'ref_hash', how = 'left')

In [63]:
eventsInstallsVentanaUno = pd.merge(eventsInstallsVentanaUno, cantidadDeInstallsPorIdVentanaUno,\
                                    on = 'ref_hash', how = 'left')
eventsInstallsVentanaDos = pd.merge(eventsInstallsVentanaDos, cantidadDeInstallsPorIdVentanaDos,\
                                    on = 'ref_hash', how = 'left')
eventsInstallsVentanaTres = pd.merge(eventsInstallsVentanaTres, cantidadDeInstallsPorIdVentanaTres,\
                                    on = 'ref_hash', how = 'left')
eventsInstallsVentanaCuatro = pd.merge(eventsInstallsVentanaCuatro, cantidadDeInstallsPorIdVentanaCuatro,\
                                    on = 'ref_hash', how = 'left')
eventsInstallsVentanaCinco = pd.merge(eventsInstallsVentanaCinco, cantidadDeInstallsPorIdVentanaCinco,\
                                    on = 'ref_hash', how = 'left')

In [64]:
eventsInstallsVentanaUno.head(2)

,index,date,event_id,ref_hash,application_id,attributed,device_city,event_uuid,carrier,kind,device_os,wifi,connection_type,device_language,installed,proporcion_wifi,sum,tiempo_prom_entre_installs,cantidad_eventos,cantidad_installs
0,2130678,178969.12,0,5857744372586891366,210,False,NaN,5b506964-5f47-4b28-a8c2-8a92d6c23379,NaN,5.882882e+18,NaN,False,NaN,3.301378e+18,0.0,0.0,1,NaN,3,0.0
1,2130680,178969.34,1,7642521036780133571,210,False,NaN,f1fb9d15-1a7b-4116-8d3b-c4c403e197e2,NaN,4.017674e+18,NaN,False,NaN,NaN,0.0,0.0,1,NaN,95,0.0


### Ahora pasando el tiempo de los que no llevan install a 3 dias.

In [65]:
eventsInstallsVentanaUno.loc[eventsInstallsVentanaUno.cantidad_installs == 0, 'date'] = 259200
eventsInstallsVentanaDos.loc[eventsInstallsVentanaDos.cantidad_installs == 0, 'date'] = 259200
eventsInstallsVentanaTres.loc[eventsInstallsVentanaTres.cantidad_installs == 0, 'date'] = 259200
eventsInstallsVentanaCuatro.loc[eventsInstallsVentanaCuatro.cantidad_installs == 0, 'date'] = 259200
eventsInstallsVentanaCinco.loc[eventsInstallsVentanaCinco.cantidad_installs == 0, 'date'] = 259200

### Vamos a quedarnos con una unica ocurrencia de cada ref_hash, la primera de ellos. Por eso hacemos un sort_values por ref_hash y por date.

### Lo hacemos en un nuevo dataframe para tener el anterior a disposicion para armar nuevos modelos

In [66]:
eventsInstallsVentanaUnoRegresion = eventsInstallsVentanaUno.sort_values(['ref_hash', 'date'], ascending = [True, True])
eventsInstallsVentanaDosRegresion = eventsInstallsVentanaDos.sort_values(['ref_hash', 'date'], ascending = [True, True])
eventsInstallsVentanaTresRegresion = eventsInstallsVentanaTres.sort_values(['ref_hash', 'date'], ascending = [True, True])
eventsInstallsVentanaCuatroRegresion = eventsInstallsVentanaCuatro.sort_values(['ref_hash', 'date'], ascending = [True, True])
eventsInstallsVentanaCincoRegresion = eventsInstallsVentanaCinco.sort_values(['ref_hash', 'date'], ascending = [True, True])

In [67]:
eventsInstallsVentanaUnoRegresion = eventsInstallsVentanaUnoRegresion.drop_duplicates(subset = ['ref_hash'])
eventsInstallsVentanaDosRegresion = eventsInstallsVentanaDosRegresion.drop_duplicates(subset = ['ref_hash'])
eventsInstallsVentanaTresRegresion = eventsInstallsVentanaTresRegresion.drop_duplicates(subset = ['ref_hash'])
eventsInstallsVentanaCuatroRegresion = eventsInstallsVentanaCuatroRegresion.drop_duplicates(subset = ['ref_hash'])
eventsInstallsVentanaCincoRegresion = eventsInstallsVentanaCincoRegresion.drop_duplicates(subset = ['ref_hash'])

In [68]:
eventsInstallsVentanaUnoRegresion.head(2)

,index,date,event_id,ref_hash,application_id,attributed,device_city,event_uuid,carrier,kind,device_os,wifi,connection_type,device_language,installed,proporcion_wifi,sum,tiempo_prom_entre_installs,cantidad_eventos,cantidad_installs
269132,14701,76310.326,23,41863526108385,121,False,NaN,2f8be0cc-297e-4c9c-a097-1096aa5824b5,NaN,6.081165e+18,NaN,False,NaN,6.977049e+18,1.0,0.0,1,326.205,88,2.0
109661,63891,259200.000,23,161514654074162,121,False,NaN,45d306f1-3195-4e37-b115-126b46481316,NaN,6.081165e+18,NaN,True,NaN,6.977049e+18,0.0,100.0,1,NaN,8,0.0


In [69]:
eventsInstallsVentanaUnoRegresionA = eventsInstallsVentanaUnoRegresion[['proporcion_wifi', 'cantidad_eventos',\
                                                                       'cantidad_installs', 'tiempo_prom_entre_installs',\
                                                                       'date']]
eventsInstallsVentanaDosRegresionA = eventsInstallsVentanaDosRegresion[['proporcion_wifi', 'cantidad_eventos',\
                                                                       'cantidad_installs', 'tiempo_prom_entre_installs',\
                                                                       'date']]
eventsInstallsVentanaTresRegresionA = eventsInstallsVentanaTresRegresion[['proporcion_wifi', 'cantidad_eventos',\
                                                                       'cantidad_installs', 'tiempo_prom_entre_installs',\
                                                                       'date']]
eventsInstallsVentanaCuatroRegresionA = eventsInstallsVentanaCuatroRegresion[['proporcion_wifi', 'cantidad_eventos',\
                                                                       'cantidad_installs', 'tiempo_prom_entre_installs',\
                                                                       'date']]
eventsInstallsVentanaCincoRegresionA = eventsInstallsVentanaCincoRegresion[['proporcion_wifi', 'cantidad_eventos',\
                                                                       'cantidad_installs', 'tiempo_prom_entre_installs',\
                                                                       'date']]

### Aca se da la posibilidad de llenar los NaN con un tiempo de 3 días, es decir 259200 segundos. Lo haremos con y sin eso, en el fit xgboost. Para numpy, es necesario tener valores que no sean NaN.

In [70]:
eventsInstallsVentanaUnoRegresionA[['tiempo_prom_entre_installs']] = \
eventsInstallsVentanaUnoRegresionA[['tiempo_prom_entre_installs']].fillna(value = 259200)
eventsInstallsVentanaDosRegresionA[['tiempo_prom_entre_installs']] = \
eventsInstallsVentanaDosRegresionA[['tiempo_prom_entre_installs']].fillna(value = 259200)
eventsInstallsVentanaTresRegresionA[['tiempo_prom_entre_installs']] = \
eventsInstallsVentanaTresRegresionA[['tiempo_prom_entre_installs']].fillna(value = 259200)
eventsInstallsVentanaCuatroRegresionA[['tiempo_prom_entre_installs']] = \
eventsInstallsVentanaCuatroRegresionA[['tiempo_prom_entre_installs']].fillna(value = 259200)
eventsInstallsVentanaCincoRegresionA[['tiempo_prom_entre_installs']] = \
eventsInstallsVentanaCincoRegresionA[['tiempo_prom_entre_installs']].fillna(value = 259200)

/Users/yorickvinesse/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [71]:
eventsInstallsVentanaUnoRegresionA.head(3)

,proporcion_wifi,cantidad_eventos,cantidad_installs,tiempo_prom_entre_installs,date
269132,0.0,88,2.0,326.205,76310.326
109661,100.0,8,0.0,259200.000,259200.000
326585,0.0,13,1.0,60042.839,60042.839


In [72]:
eventsInstallsVentanaUnoRegresionA.count()

proporcion_wifi               166002
cantidad_eventos              166002
cantidad_installs             166002
tiempo_prom_entre_installs    166002
date                          166002
dtype: int64

### Paso necesario para ingresar datos a fitters de scikit y de XGBoost

In [73]:
arrayVentanaUnoRegresionA = eventsInstallsVentanaUnoRegresionA.values
arrayVentanaDosRegresionA = eventsInstallsVentanaDosRegresionA.values
arrayVentanaTresRegresionA = eventsInstallsVentanaTresRegresionA.values
arrayVentanaCuatroRegresionA = eventsInstallsVentanaCuatroRegresionA.values
arrayVentanaCincoRegresionA = eventsInstallsVentanaCincoRegresionA.values

In [76]:
X_VentanaUnoRegresionA = arrayVentanaUnoRegresionA[:, :-1]
y_VentanaUnoRegresionA = arrayVentanaUnoRegresionA[:, -1]
X_VentanaDosRegresionA = arrayVentanaDosRegresionA[:, :-1]
y_VentanaDosRegresionA = arrayVentanaDosRegresionA[:, -1]
X_VentanaTresRegresionA = arrayVentanaTresRegresionA[:, :-1]
y_VentanaTresRegresionA = arrayVentanaTresRegresionA[:, -1]
X_VentanaCuatroRegresionA = arrayVentanaCuatroRegresionA[:, :-1]
y_VentanaCuatroRegresionA = arrayVentanaCuatroRegresionA[:, -1]
X_VentanaCincoRegresionA = arrayVentanaCincoRegresionA[:, :-1]
y_VentanaCincoRegresionA = arrayVentanaCincoRegresionA[:, -1]

## Con la regresion lineal de scikit

In [96]:
regVentanaUnoA = LinearRegression().fit(X_VentanaUnoRegresionA, y_VentanaUnoRegresionA)
regVentanaDosA = LinearRegression().fit(X_VentanaDosRegresionA, y_VentanaDosRegresionA)
regVentanaTresA = LinearRegression().fit(X_VentanaTresRegresionA, y_VentanaTresRegresionA)
regVentanaCuatroA = LinearRegression().fit(X_VentanaCuatroRegresionA, y_VentanaCuatroRegresionA)
regVentanaCincoA = LinearRegression().fit(X_VentanaCincoRegresionA, y_VentanaCincoRegresionA)

In [78]:
regVentanaUnoA.score(X_VentanaUnoRegresionA, y_VentanaUnoRegresionA)

0.9132174973693319

### El score para si mismo esta alto, la predicción esta bastante buena.

In [98]:
regVentanaUnoA.coef_

array([-5.85696566e+00, -1.41389940e+01, -3.73757482e+03,  9.33396781e-01])

In [100]:
regVentanaDosA.coef_

array([-8.51433241e+00, -1.38105817e+01, -3.80749040e+03,  9.24202176e-01])

### En esa regresión, se da mucha importancia al feature 3, es decir la cantidad de installs.

In [80]:
prediccionVentanaUnoSobreVentanaDosA = regVentanaUnoA.predict(X_VentanaDosRegresionA)

In [102]:
scoresSobreVentanaSiguienteRegresionA = [regVentanaUnoA.score(X_VentanaDosRegresionA, y_VentanaDosRegresionA),\
                                    regVentanaDosA.score(X_VentanaTresRegresionA, y_VentanaTresRegresionA),\
                                     regVentanaTresA.score(X_VentanaCuatroRegresionA, y_VentanaCuatroRegresionA),\
                                     regVentanaCuatroA.score(X_VentanaCincoRegresionA, y_VentanaCincoRegresionA)]

In [103]:
scoresSobreVentanaSiguienteRegresionA

[0.9020125359249261, 0.9063672310274357, 0.919034337499054, 0.9191289490852438]

## Con XGBoost

In [104]:
xgbVentanaUnoRegresionA = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, objective='reg:squarederror')
xgbVentanaDosRegresionA = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, objective='reg:squarederror')
xgbVentanaTresRegresionA = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, objective='reg:squarederror')
xgbVentanaCuatroRegresionA = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, objective='reg:squarederror')
xgbVentanaCincoRegresionA = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, objective='reg:squarederror')

In [105]:
xgbVentanaUnoRegresionA.fit(X_VentanaUnoRegresionA, y_VentanaUnoRegresionA)
xgbVentanaDosRegresionA.fit(X_VentanaDosRegresionA, y_VentanaDosRegresionA)
xgbVentanaTresRegresionA.fit(X_VentanaTresRegresionA, y_VentanaTresRegresionA)
xgbVentanaCuatroRegresionA.fit(X_VentanaCuatroRegresionA, y_VentanaCuatroRegresionA)
xgbVentanaCincoRegresionA.fit(X_VentanaCincoRegresionA, y_VentanaCincoRegresionA)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.08, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=0.75, verbosity=1)

In [106]:
predictVentanaUnoSobreVentanaDos = xgbVentanaUnoRegresionA.predict(X_VentanaDosRegresionA)
predictVentanaDosSobreVentanaTres = xgbVentanaDosRegresionA.predict(X_VentanaTresRegresionA)
predictVentanaTresSobreVentanaCuatro = xgbVentanaTresRegresionA.predict(X_VentanaCuatroRegresionA)
predictVentanaCuatroSobreVentanaCinco = xgbVentanaCuatroRegresionA.predict(X_VentanaCincoRegresionA)

In [107]:
xgbScoresSobreVentanaSiguienteRegresionA = [explained_variance_score(predictVentanaUnoSobreVentanaDos,y_VentanaDosRegresionA),\
                                           explained_variance_score(predictVentanaDosSobreVentanaTres,y_VentanaTresRegresionA),\
                                           explained_variance_score(predictVentanaTresSobreVentanaCuatro,y_VentanaCuatroRegresionA),\
                                           explained_variance_score(predictVentanaCuatroSobreVentanaCinco,y_VentanaCincoRegresionA)]


In [94]:
print(explained_variance_score(predictVentanaUnoSelf,y_VentanaUnoRegresionA))

0.9674227884767749


In [108]:
xgbScoresSobreVentanaSiguienteRegresionA

[0.950279878779768, 0.9527291356692622, 0.9530554877670591, 0.967660875441266]

### Muy buenos resultados con el algoritmo XGB, con un accuracy de 0.95 para la ventana dos con el modelo 1.

### Ahora recompilamos nuestros resultados con los device_id correspondientes. El calculo con XGBoost sera él usado para el target

In [164]:
dfPredictVentanaUnoSobreVentanaDos = pd.DataFrame(predictVentanaUnoSobreVentanaDos, columns = ['prediccion_1'])
dfPredictVentanaDosSobreVentanaTres = pd.DataFrame(predictVentanaDosSobreVentanaTres, columns = ['prediccion_2'])
dfPredictVentanaTresSobreVentanaCuatro = pd.DataFrame(predictVentanaTresSobreVentanaCuatro, columns = ['prediccion_3'])
dfPredictVentanaCuatroSobreVentanaCinco = pd.DataFrame(predictVentanaCuatroSobreVentanaCinco, columns = ['prediccion_4'])

In [165]:
dfPredictVentanaDosSobreVentanaTres.head()

,prediccion_2
0,259110.156250
1,259110.156250
2,158814.703125
3,259110.156250
4,259110.156250


In [174]:
dfPrediccionVentanaDosRegresionA = eventsInstallsVentanaDosRegresion[['ref_hash']]
dfPrediccionVentanaTresRegresionA = eventsInstallsVentanaTresRegresion[['ref_hash']]
dfPrediccionVentanaCuatroRegresionA = eventsInstallsVentanaCuatroRegresion[['ref_hash']]
dfPrediccionVentanaCincoRegresionA = eventsInstallsVentanaCincoRegresion[['ref_hash']]

In [175]:
dfPrediccionVentanaDosRegresionA = dfPrediccionVentanaDosRegresionA.reset_index(drop = True)
dfPrediccionVentanaTresRegresionA = dfPrediccionVentanaTresRegresionA.reset_index(drop = True)
dfPrediccionVentanaCuatroRegresionA = dfPrediccionVentanaCuatroRegresionA.reset_index(drop = True)
dfPrediccionVentanaCincoRegresionA = dfPrediccionVentanaCincoRegresionA.reset_index(drop = True)

In [176]:
dfPrediccionVentanaTresRegresionA.head()

,ref_hash
0,40621409780134
1,41863526108385
2,161514654074162
3,168103949904656
4,186034136943920


In [177]:
dfPrediccionVentanaDosRegresionA = pd.concat([dfPrediccionVentanaDosRegresionA,\
                                             dfPredictVentanaUnoSobreVentanaDos], axis = 1)
dfPrediccionVentanaTresRegresionA = pd.concat([dfPrediccionVentanaTresRegresionA,\
                                             dfPredictVentanaDosSobreVentanaTres], axis = 1)
dfPrediccionVentanaCuatroRegresionA = pd.concat([dfPrediccionVentanaCuatroRegresionA,\
                                             dfPredictVentanaTresSobreVentanaCuatro], axis = 1)
dfPrediccionVentanaCincoRegresionA = pd.concat([dfPrediccionVentanaCincoRegresionA,\
                                             dfPredictVentanaCuatroSobreVentanaCinco], axis = 1)

In [178]:
dfPrediccionVentanaCincoRegresionA.head(5)

,ref_hash,prediccion_4
0,40621409780134,259113.359375
1,69039685746313,259113.359375
2,90072729247980,259113.359375
3,161514654074162,100249.804688
4,168103949904656,259113.359375


### Ahora mergeamos los cuatro dataframes para tener el promedio asociado a cada modelo en una columna distinta. Despues de so, haremos un promedio de las predicciones y lo tomaremos como Survival Time para un install.

In [291]:
dfPrediccionSc = pd.merge(dfPrediccionVentanaDosRegresionA, dfPrediccionVentanaTresRegresionA,\
                        on = 'ref_hash', how = 'outer')

In [292]:
dfPrediccionSc = pd.merge(dfPrediccionSc, dfPrediccionVentanaCuatroRegresionA,\
                        on = 'ref_hash', how = 'outer')
dfPrediccionSc = pd.merge(dfPrediccionSc, dfPrediccionVentanaCincoRegresionA,\
                        on = 'ref_hash', how = 'outer')

In [235]:
len(dfPrediccionSc)

273948

In [293]:
dfPrediccionSc.head(5)

,ref_hash,prediccion_1,prediccion_2,prediccion_3,prediccion_4
0,40621409780134,259119.390625,259110.156250,259114.703125,259113.359375
1,41863526108385,259119.390625,259110.156250,NaN,NaN
2,161514654074162,259126.875000,158814.703125,193308.062500,100249.804688
3,360710529886978,259119.390625,259110.156250,259114.703125,259113.359375
4,365882020742330,259119.390625,259110.156250,259114.703125,259113.359375


In [294]:
dfPrediccionSc = dfPrediccionSc.fillna(value = 259200)

In [295]:
dfPrediccionSc.head()

,ref_hash,prediccion_1,prediccion_2,prediccion_3,prediccion_4
0,40621409780134,259119.390625,259110.156250,259114.703125,259113.359375
1,41863526108385,259119.390625,259110.156250,259200.000000,259200.000000
2,161514654074162,259126.875000,158814.703125,193308.062500,100249.804688
3,360710529886978,259119.390625,259110.156250,259114.703125,259113.359375
4,365882020742330,259119.390625,259110.156250,259114.703125,259113.359375


In [296]:
dfPrediccionSc['prediccion_promedia'] = 0.25*(dfPrediccionSc.prediccion_1 + dfPrediccionSc.prediccion_2 +\
dfPrediccionSc.prediccion_3 + dfPrediccionSc.prediccion_4)

In [297]:
dfPrediccionSc.head()

,ref_hash,prediccion_1,prediccion_2,prediccion_3,prediccion_4,prediccion_promedia
0,40621409780134,259119.390625,259110.156250,259114.703125,259113.359375,259114.406250
1,41863526108385,259119.390625,259110.156250,259200.000000,259200.000000,259157.390625
2,161514654074162,259126.875000,158814.703125,193308.062500,100249.804688,177874.859375
3,360710529886978,259119.390625,259110.156250,259114.703125,259113.359375,259114.406250
4,365882020742330,259119.390625,259110.156250,259114.703125,259113.359375,259114.406250


In [298]:
dfPrediccionSc = dfPrediccionSc[['ref_hash', 'prediccion_promedia']]

### Ahora vamos a completar con todos los ref_hash que nunca aparecieron en nuestro estudio pero que son parte del estudio.

In [299]:
dfPrediccionSc['ref_hash'] = dfPrediccionSc.ref_hash.astype(str) + '_sc'

In [300]:
dfPrediccionSc.head(3)

,ref_hash,prediccion_promedia
0,40621409780134_sc,259114.406250
1,41863526108385_sc,259157.390625
2,161514654074162_sc,177874.859375


In [255]:
target = pd.read_csv('./target_competencia_ids.csv')

In [191]:
target.dtypes

ref_hash    object
obj          int64
dtype: object

In [256]:
target.head(3)

,ref_hash,obj
0,1000169251625791246_sc,0
1,1000169251625791246_st,0
2,1000395625957344683_sc,0


In [316]:
targetUpdated = pd.merge(target, dfPrediccionSc, on = 'ref_hash', how = 'left')

In [317]:
targetUpdated.head(15)

,ref_hash,obj,prediccion_promedia
0,1000169251625791246_sc,0,259114.406250
1,1000169251625791246_st,0,NaN
2,1000395625957344683_sc,0,161908.000000
3,1000395625957344683_st,0,NaN
4,1003027494996471685_sc,0,182100.312500
5,1003027494996471685_st,0,NaN
6,1006670001679961544_sc,0,195578.828125
7,1006670001679961544_st,0,NaN
8,1007573308966476713_sc,0,259114.406250
9,1007573308966476713_st,0,NaN


### Si nunca apareció en nuestro analisis, le ponemos un valor de tres dias.

In [318]:
targetUpdated =  targetUpdated.fillna(value = 259200)

In [319]:
targetUpdated = targetUpdated.drop(columns = ['obj'])
targetUpdated.columns = ['ref_hash', 'obj']

### Consideremos que los valores muy cerca de 259200 segundos son una aproximación para tres días asi las cambiamos.

In [320]:
targetUpdated.loc[targetUpdated.obj > 255000, 'obj'] = 259200

### Dejando los valores de St(d) a 0 por ahora

In [321]:
targetUpdated.loc[targetUpdated.index % 2 != 0, 'obj'] = 0

In [322]:
targetUpdated.dtypes

ref_hash     object
obj         float64
dtype: object

In [323]:
targetUpdated['obj'] = targetUpdated.obj.round().astype(int)

In [324]:
targetUpdated.head(20)

,ref_hash,obj
0,1000169251625791246_sc,259200
1,1000169251625791246_st,0
2,1000395625957344683_sc,161908
3,1000395625957344683_st,0
4,1003027494996471685_sc,182100
5,1003027494996471685_st,0
6,1006670001679961544_sc,195579
7,1006670001679961544_st,0
8,1007573308966476713_sc,259200
9,1007573308966476713_st,0


## Aca tenemos nuestra primera prediccion para las conversiones.

In [325]:
targetUpdated.to_csv('targetConSc.csv', encoding='utf-8', index=False)